In [4]:
#! /usr/bin/python2.7
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import feedparser
import sys
import requests
from imp import reload

reload(sys)
sys.setdefaultencoding('utf-8')

rss = "http://www.boannews.com/media/news_rss.xml?kind=1"  #뉴스 rss 주소
filename = "result.txt"  #결과를 저장할 파일 이름 (폴더에 저장하고 싶으면. 폴더를 미리 만들어두고 "폴더명/파일명.txt" 해주면 됨)


### rss 피드 읽어오는 함수
def spider(max_page):
    page = 0 #읽어온 페이지 수를 카운팅하기 위한 변수
    out = open(filename, 'w') #쓰기 위한 파일 열기
    
    while page < max_page: #page가 max_page보다 작을 때까지 수행
        post_list = feedparser.parse(rss) #현재 rss 페이지 내용을 parsing해서 feed_list에 저장

        for post in post_list.entries: #현재 페이지에 있는 포스트 갯수만큼 반복
            get_single_article(out, post.link) #쓰고자하는 파일명과 포스트의 link주소를 get_single_article 함수에 넘겨줌
                
        page += 1 #읽어온 페이지 갯수 증가
    
    out.close() #파일 닫기

    
### 링크를 타고가서 해당 기사의 본문을 저장하는 함수
def get_single_article(out, link_url):
    response = requests.get(link_url) #link_url 주소로 http 요청해서 그 결과를 response에 저장
    plain_text = response.text # 위의 결과에서 text 문자만 취함
    soup = BeautifulSoup(plain_text, 'lxml') #'lxml'은 xml같은 마크업 언어 parser 중 하나인듯
    
    time = soup.find_all(id='news_util01') #보안뉴스 페이지의 HTML 요소에서 시간을 나타내는 태그를 찾음 "news_util01"
    print >> out, time[0].text #out(저장할 텍스트파일 변수)에 시간텍스트 쓰기

    for contents in soup.find_all(id='news_content'): #보안뉴스 페이지의 HTML 요소에서 본문을 나타내는 태그를 찾음 "news_content"
        print >> out, contents.text #out(저장할 텍스트파일 변수)에 기사본문텍스트 쓰기

    end_mark = "" #공백 문자열 선언
    end_mark += "="*200 #구분자 저장  ex)"==================="
    print >> out, end_mark #out(저장할 텍스트파일 변수)에 구분자 쓰기
        
    
    
########## 프로그램 시작함수 부르는 부분 ###########
spider(1) #crawling하고자하는 rss페이지수를 파라미터로 넘겨주면 됨